<a href="https://colab.research.google.com/github/xxxcrttt/MLDL/blob/main/CS224W/Node2vec_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Node2Vec 官方作者 Aditya Grover 代码讲解

In [40]:
# 导入工具包
import warnings 
warnings.filterwarnings('ignore')
import argparse
import numpy as np 
import networkx as nx 
from gensim.models import word2vec
import random

import matplotlib.pyplot as plt
%matplotlib inline 

In [44]:
# 读取命令行参数
def parse_args():
	'''
	Parses the node2vec arguments.
	'''
  # 使用parser加载信息
	parser = argparse.ArgumentParser(description="Run node2vec.")
  # 输入文件：邻接表
	parser.add_argument('--input', nargs='?', default='graph/karate.edgelist',
	                    help='Input graph path')
  # 输出文件：节点嵌入表
	parser.add_argument('--output', nargs='?', default='emb/karate.emb',
	                    help='Embeddings path')
  # embedding 嵌入向量维度
	parser.add_argument('--dimensions', type=int, default=128,
	                    help='Number of dimensions. Default is 128.')
  # 随机游走序列长度
	parser.add_argument('--walk-length', type=int, default=80,
	                    help='Length of walk per source. Default is 80.')
  # 每个节点生成随机游走序列次数
	parser.add_argument('--num-walks', type=int, default=10,
	                    help='Number of walks per source. Default is 10.')
  # word2vec 窗口大小，word2vec 参数
	parser.add_argument('--window-size', type=int, default=10,
                    	help='Context size for optimization. Default is 10.')
  # SGD 优化时epoch 数量，word2vec 参数
	parser.add_argument('--iter', default=1, type=int,
                      help='Number of epochs in SGD')
  # 并行化核数，word2vec 参数
	parser.add_argument('--workers', type=int, default=8,
	                    help='Number of parallel workers. Default is 8.')
  # 参数 p
	parser.add_argument('--p', type=float, default=1,
	                    help='Return hyperparameter. Default is 1.')
  # 参数 q
	parser.add_argument('--q', type=float, default=1,
	                    help='Inout hyperparameter. Default is 1.')
  # 连接是否带有权重
	parser.add_argument('--weighted', dest='weighted', action='store_true',
	                    help='Boolean specifying (un)weighted. Default is unweighted.')
	parser.add_argument('--unweighted', dest='unweighted', action='store_false')
	parser.set_defaults(weighted=False)
  # 有向图 or 无向图
	parser.add_argument('--directed', dest='directed', action='store_true',
	                    help='Graph is (un)directed. Default is undirected.')
	parser.add_argument('--undirected', dest='undirected', action='store_false')
	parser.set_defaults(directed=False)

	return parser.parse_args(args=[])
 
args = parse_args()

In [45]:
args

Namespace(input='graph/karate.edgelist', output='emb/karate.emb', dimensions=128, walk_length=80, num_walks=10, window_size=10, iter=1, workers=8, p=1, q=1, weighted=False, unweighted=True, directed=False, undirected=True)

## 载入图

In [ ]:
# 连接带权重
if args.weighted:
  G = nx.read_edgelist(args.input, nodetype=int, data=(('weight', float),),create_using=nx.DiGraph())

# 连接不带权重 -- 给每个连接随机赋予一个权重
else:
  G = nx.read_edgelist(args.input, nodetype=int, create_using=nx.DiGraph())
  for edge in G.edges():
    G[edge[0]][edge[1]]['weight'] = np.abs(np.random.randn())

# 无向图
if not args.directed:
  G = G.to_undirected()

## Alias sampling 

In [51]:
def alias_setup(probs):
	'''
	Compute utility lists for non-uniform sampling from discrete distributions.
	Refer to https://hips.seas.harvard.edu/blog/2013/03/03/the-alias-method-efficient-sampling-with-many-discrete-outcomes/
	for details
	'''
	K = len(probs)
	q = np.zeros(K)
	J = np.zeros(K, dtype=np.int)

	smaller = []
	larger = []

  # 将各个概率分成两组, 一组的概率值 > 1, 另一组的概率值 < 1
	for kk, prob in enumerate(probs):
   # 每类事件的概率 X 事件个数
	    q[kk] = K*prob 
      # 判定 "劫富" 和 "济贫" 的对象
	    if q[kk] < 1.0:
	        smaller.append(kk)
	    else:
	        larger.append(kk)
         
  # 使用谈心算法，将概率值 < 1 的不断填满
  # pseudo code step 3
	while len(smaller) > 0 and len(larger) > 0:
	    small = smaller.pop()
	    large = larger.pop()

	    J[small] = large
      # 更新概率值
	    q[large] = q[large] + q[small] - 1.0
	    if q[large] < 1.0:
	        smaller.append(large)
	    else:
	        larger.append(large)

	return J, q

In [ ]:
def alias_draw(J, q):
	'''
	Draw sample from a non-uniform discrete distribution using alias sampling.
	'''
	K = len(J) # 事件个数

	kk = int(np.floor(np.random.rand()*K))
	if np.random.rand() < q[kk]:
	    return kk # 取自己本来对应的事件
	else:
	    return J[kk] # 取alias 事件

In [ ]:
def get_alias_edge(self, src, dst):
		'''
		Get the alias edge setup lists for a given edge.
		'''
		G = self.G
		p = self.p
		q = self.q

		unnormalized_probs = []
    # 论文3.2核心算法，计算各条边的转移权重
		for dst_nbr in sorted(G.neighbors(dst)):
			if dst_nbr == src:
				unnormalized_probs.append(G[dst][dst_nbr]['weight']/p)
			elif G.has_edge(dst_nbr, src):
				unnormalized_probs.append(G[dst][dst_nbr]['weight'])
			else:
				unnormalized_probs.append(G[dst][dst_nbr]['weight']/q)
    # 归一化各条边的转移权重
		norm_const = sum(unnormalized_probs)
		normalized_probs =  [float(u_prob)/norm_const for u_prob in unnormalized_probs]

    # 执行 alias samping
		return alias_setup(normalized_probs)

## 生成一条随机游走序列

In [ ]:
def node2vec_walk(self, walk_length, start_node):
		'''
		Simulate a random walk starting from start node.
    从指定的起始节点，生成一个随机游走序列
		'''

		G = self.G
		alias_nodes = self.alias_nodes
		alias_edges = self.alias_edges

    # 上一步计算出的 alias table, 完成O(1)的采样
		walk = [start_node]

    # 直到生成长度为 walk_length 的节点序列为止
		while len(walk) < walk_length:
			cur = walk[-1]
      # 对邻居节点排序, 目的是和 alias table 计算时的顺序对应起来
			cur_nbrs = sorted(G.neighbors(cur))
			if len(cur_nbrs) > 0:
        # 节点序列只有一个节点的情况
				if len(walk) == 1:
					walk.append(cur_nbrs[alias_draw(alias_nodes[cur][0], alias_nodes[cur][1])])
     # 节点序列大于一个节点的情况
				else:
          # 看前一个节点，prev 是节点 t
					prev = walk[-2]
					next = cur_nbrs[alias_draw(alias_edges[(prev, cur)][0], 
						alias_edges[(prev, cur)][1])]
					walk.append(next)
			else:
				break

		return walk

## 采样得到所有随机游走序列

In [ ]:
def simulate_walks(self, num_walks, walk_length):
		'''
		Repeatedly simulate random walks from each node.
    图中每个节点作为起始节点，生成 num_walk 个随机游走序列
		'''
		G = self.G
		walks = []
		nodes = list(G.nodes())
		print 'Walk iteration:'
		for walk_iter in range(num_walks):
			print str(walk_iter+1), '/', str(num_walks)
   # 打乱节点顺序
			random.shuffle(nodes)
			for node in nodes:
				walks.append(self.node2vec_walk(walk_length=walk_length, start_node=node))

		return walks